In [ ]:
from calc_self_corr import *
from machine_lib_output import *

username = ""
password = ""

# 登录
sess = sign_in(username, password)

In [9]:


# 增量下载数据
download_data(sess, flag_increment=True)

# 加载数据， 如果需要使用不同的标签，可以传入 tag 参数， 例如 tag='PPAC' 或 tag='SelfCorr'
os_alpha_ids, os_alpha_rets = load_data()

从偏移 0 到 30 获取 alpha
新下载的 alpha 数量: 1, 总 alpha 数量: 213


In [10]:
regions = ['USA', 'CHN', 'GLB', 'EUR', 'ASI']
stone_bag_atom = {}
stone_bag_hp = {}
ppa_tracker_atom_all = []
ppa_tracker_hp_all = []
date = "2025-06-08"
file_path = f"output/simulated_alphas.csv.{date}"
# 假设要过滤的关键字列表
keywords_to_exclude = ['mdl230']
for region in regions:
    ppa_tracker_atom = get_alphas_from_csv(file_path, 1.0, 0.3, "submit", region, single_data_set_filter=True)
    ppa_tracker_atom_all.extend(ppa_tracker_atom)
    ppa_tracker_hp = get_alphas_from_csv(file_path, 1.5, 0.8, "submit", region, single_data_set_filter=None)
    ppa_tracker_hp_all.extend(ppa_tracker_hp)
    # 修改后的代码
    region_bag_atom = [alpha[0] for alpha in ppa_tracker_atom if not any(keyword in alpha[1] for keyword in keywords_to_exclude)]
    region_bag_hp = [alpha[0] for alpha in ppa_tracker_hp if not any(keyword in alpha[1] for keyword in keywords_to_exclude)]
    stone_bag_hp[region] = region_bag_hp
    stone_bag_atom[region] = region_bag_atom

# 打印结果
for region in regions:
    print(f"{region}: {len(stone_bag_atom[region])}  atom alphas, {len(stone_bag_hp[region])}  high performance alphas")
    print(stone_bag_atom[region])
    print(stone_bag_hp[region])
print(f"Total atom alphas: {len(ppa_tracker_atom_all)}")
print(f"Total high performance alphas: {len(ppa_tracker_hp_all)}")

USA: 9  atom alphas, 497  high performance alphas
['LmLGObv', 'GKrloOP', '2XrOaGP', '3kqXKmZ', 'OM0LZxd', 'x2OXrZW', 'brGd6rp', 'K5302Ax', 'Ar97wmR']
['nepY1Wx', '3kq3M0N', 'ZdYqY70', 'm0LJLm2', 'OM0q0XY', '1lY1eJm', 'XvgPG8m', 'aQkM611', 'm0LJpaX', 'E85Y1z1', 'Ar1YM1e', 'd35gApX', 'RlRqn3d', 'LmLqJJ2', '3kq3LxO', '7v8093L', 'brlVErK', 'jP2WmGO', 'rNKEXn8', 'w2pOneY', '9dqba7K', 'm0LJrjx', '9dqbax1', 'QNPq2xG', 'd35gnAY', 'NJ1qnoq', 'x2A9nzw', 'pMwo7lv', '9dqbREe', 'LmLqREa', 'LmLqR3v', 'z2qQWEG', 'OM0qpYb', 'ggJw1vO', 'MePqLGa', '7v80no1', 'YEPng9w', '3kq3eXX', 'Ar1Y7Ql', 'PY0qOmw', 'aQkMEmw', 'w2pONRd', 'pMwoWR6', '9dqbmR9', 'brlVnjR', '0Nm96xq', 'Vmkq5LM', 'w2pOPQp', 'K5PqQQE', 'brlVgYN', 'VmkqLoA', 'pMwokJv', '8E87WZq', 'ZdYq8wY', '7v80505', 'pMwo2PX', '5A83mgo', 'w2pOneY', 'l5e2pQ7', '1lYLkXk', 'K5P60Vg', 'qYMd1wO', 'd35gnAY', '9dqK1Jq', 'm0LJrjx', 'ZdYx3bZ', 'XvgLX10', 'm0LJpaX', 'E85Y1z1', 'rNKLzW8', '0NmdKZ1', 'E85bg51', 'w2p15wp', 'd35mngK', 'K5P6n8x', 'PY06n3W', 'QNPq2xG', 'G

In [11]:
def process_stone_bag(stone_bag, regions, output_file_path, sess, ppa_tracker=None, tag='PPAC'):
    """
    对 stone_bag 中每个 region 的 alpha_id 进行自相关性筛选，并写入 CSV 文件。
    新增 ppa_tracker 参数用于查询 alpha_id 对应的 pyramids 信息。
    优化：将 ppa_tracker 转换为字典，使查询时间复杂度降为 O(1)。
    """
    valid_alphas_by_region = {}

    # 优化：将 ppa_tracker 转换为字典 {alpha_id: pyramids}
    ppa_tracker_dict = {}
    if ppa_tracker is not None:
        for item in ppa_tracker:
            alpha_id = item[0]
            pyramids = item[7]  # pyramids 是第7个元素
            ppa_tracker_dict[alpha_id] = pyramids
    print(ppa_tracker_dict)

    # 确保输出路径存在
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        # 修改表头，增加 pyramids 列
        writer.writerow(["region", "alpha_id", "self_corr", "pyramids"])  

        for region in regions:
            region_stone_bag = stone_bag.get(region, [])
            valid_alphas = []
            idx = 0

            for alpha_id in region_stone_bag:
                idx += 1

                if len(region_stone_bag) > 1 and (idx % max(1, int(len(region_stone_bag) * 0.1)) == 0 or idx == 1 or idx == len(region_stone_bag)):
                    if username is not None and password is not None:
                        if not (idx == len(region_stone_bag) or len(region_stone_bag) < 100):
                            new_sess = sign_in(username, password)
                            if new_sess is None:
                                print("Failed to re-authenticate session, continuing with existing session")
                            else:
                                sess = new_sess
                                print("Session re-authenticated successfully")
                    else:
                        print("Username or password is None, skipping re-authentication and continuing with existing session")

                    print(
                        f"[{region}] Processing alpha {idx}/{len(region_stone_bag)} ({idx/len(region_stone_bag):.1%}) | "
                        f"Current: {alpha_id}"
                    )

                os_alpha_ids, os_alpha_rets = load_data(tag=tag)
                self_corr = calc_self_corr(
                    sess=sess,
                    alpha_id=alpha_id,
                    os_alpha_rets=os_alpha_rets,
                    os_alpha_ids=os_alpha_ids,
                )

                if self_corr < 0.5:
                    # 优化：直接从字典获取 pyramids，时间复杂度 O(1)
                    pyramids = ppa_tracker_dict.get(alpha_id, None)
                    
                    valid_alphas.append((alpha_id, self_corr, pyramids))
                    # 写入时包含 pyramids 信息
                    writer.writerow([region, alpha_id, self_corr, pyramids])
                else:
                    print(f"[{region}] Excluded alpha {alpha_id} with self_corr {self_corr}")

            valid_alphas_by_region[region] = valid_alphas

    # 打印所有符合条件的 alpha_id 和 pyramids 信息
    for region in regions:
        print(f"\n=== Valid Alphas for {region} from {output_file_path} ===")
        for alpha_id, self_corr, pyramids in valid_alphas_by_region.get(region, []):
            print(f"Alpha ID: {alpha_id}, Self Correlation: {self_corr}, Pyramids: {pyramids}")

    print(f"\n✅ Results saved to: {output_file_path}\n")

# -----------------------------------------
# 调用处理函数分别处理 atom 和 hp 两种 stone_bag
# -----------------------------------------
regions = ['USA', 'CHN', 'GLB', 'EUR', 'ASI']

# 你的两个 stone_bag 和 ppa_tracker
# stone_bag_atom = {...}
# stone_bag_hp = {...}
# ppa_tracker_atom = [...]
# ppa_tracker_hp = [...]

# 输出文件路径
output_file_atom = f"output/ppac_atom_gold_bag.csv.{date}"
output_file_hp = f"output/ppac_hp_gold_bag.csv.{date}"
# 分别处理，传入对应的 ppa_tracker
process_stone_bag(stone_bag_atom, regions, output_file_atom, sess, ppa_tracker_atom_all)
process_stone_bag(stone_bag_hp, regions, output_file_hp, sess, ppa_tracker_hp_all)

{'LmLGObv': [{'name': 'USA/D1/NEWS', 'multiplier': 1.2}], 'GKrloOP': [{'name': 'USA/D1/NEWS', 'multiplier': 1.2}], '2XrOaGP': [{'name': 'USA/D1/NEWS', 'multiplier': 1.2}], '3kqXKmZ': [{'name': 'USA/D1/FUNDAMENTAL', 'multiplier': 1.2}], 'OM0LZxd': [{'name': 'USA/D1/FUNDAMENTAL', 'multiplier': 1.2}], 'x2OXrZW': [{'name': 'USA/D1/ANALYST', 'multiplier': 1.2}], 'brGd6rp': [{'name': 'USA/D1/ANALYST', 'multiplier': 1.2}], 'K5302Ax': [{'name': 'USA/D1/ANALYST', 'multiplier': 1.2}], 'Ar97wmR': [{'name': 'USA/D1/ANALYST', 'multiplier': 1.2}], 'aQr5kMW': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'w280Kr6': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'vpKWemA': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'NJXYgYX': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'qYKQnQO': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'VmkvPqA': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], 'eV7lbwz': [{'name': 'EUR/D1/FUNDAMENTAL', 'multiplier': 1.1}], '1lYYx8K': [

ERROR:root:登录失败: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


Failed to re-authenticate session, continuing with existing session
[EUR] Processing alpha 39/130 (30.0%) | Current: ZdYP9Kn
8VZ21mW    0.9861
Ae8WZqw    0.3454
31O2pOP    0.3453
w3kKba1    0.2994
R9wMd3d    0.2091
vZXw2JG    0.1827
aQr5AZ9    0.1422
Y8O96KA    0.1287
8VZ2mPo    0.1204
6ZL818K    0.0984
lEYYeen    0.0947
LEw2JPM    0.0556
Zg916R0    0.0528
9833QG2    0.0518
gX5amYK    0.0285
x2Y3xJg    0.0285
2Xrd9ox    0.0260
zgnK7bG    0.0257
w3AQXq2    0.0230
3PPK2V6    0.0190
dY6ak2J    0.0137
MdnaJk6    0.0128
Ae52RYE    0.0025
eV9N0Kd   -0.0061
6ZQdKqp   -0.0092
kW0VR8O   -0.0156
dtype: float64
[EUR] Excluded alpha ZdYP9Kn with self_corr 0.9861
8VZ21mW    0.7869
31O2pOP    0.3640
Ae8WZqw    0.3404
R9wMd3d    0.2758
w3kKba1    0.2570
vZXw2JG    0.1919
aQr5AZ9    0.1418
8VZ2mPo    0.1266
MdnaJk6    0.1028
lEYYeen    0.0967
6ZL818K    0.0763
Zg916R0    0.0550
Y8O96KA    0.0546
9833QG2    0.0522
gX5amYK    0.0509
w3AQXq2    0.0495
LEw2JPM    0.0456
kW0VR8O    0.0185
x2Y3xJg    0.0157

ERROR:root:登录失败: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


Failed to re-authenticate session, continuing with existing session
[EUR] Processing alpha 65/130 (50.0%) | Current: x2A8g8m
8VZ21mW    0.8110
31O2pOP    0.3561
Ae8WZqw    0.3517
R9wMd3d    0.2775
w3kKba1    0.2724
vZXw2JG    0.2129
aQr5AZ9    0.1547
8VZ2mPo    0.1128
lEYYeen    0.1013
6ZL818K    0.0833
MdnaJk6    0.0802
Y8O96KA    0.0731
w3AQXq2    0.0616
gX5amYK    0.0515
Zg916R0    0.0461
9833QG2    0.0436
LEw2JPM    0.0432
kW0VR8O    0.0243
x2Y3xJg    0.0237
3PPK2V6    0.0165
dY6ak2J    0.0073
zgnK7bG    0.0021
eV9N0Kd   -0.0029
2Xrd9ox   -0.0055
Ae52RYE   -0.0169
6ZQdKqp   -0.0178
dtype: float64
[EUR] Excluded alpha x2A8g8m with self_corr 0.811
zgnK7bG    0.1901
2Xrd9ox    0.1512
Y8O96KA    0.1340
8VZ2mPo    0.1211
3PPK2V6    0.1073
eV9N0Kd    0.0509
lEYYeen    0.0497
Ae8WZqw    0.0457
6ZL818K    0.0429
LEw2JPM    0.0386
R9wMd3d    0.0306
Ae52RYE    0.0200
31O2pOP    0.0121
Zg916R0    0.0033
6ZQdKqp    0.0030
vZXw2JG   -0.0018
9833QG2   -0.0052
w3AQXq2   -0.0078
8VZ21mW   -0.0171


ERROR:root:登录失败: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))


Failed to re-authenticate session, continuing with existing session
[EUR] Processing alpha 91/130 (70.0%) | Current: 3kqLVYX
zgnK7bG    0.2497
3PPK2V6    0.1731
2Xrd9ox    0.1033
Y8O96KA    0.0577
LEw2JPM    0.0491
lEYYeen    0.0467
Ae8WZqw    0.0421
8VZ2mPo    0.0377
gX5amYK    0.0352
eV9N0Kd    0.0340
R9wMd3d    0.0306
x2Y3xJg    0.0254
6ZQdKqp    0.0223
vZXw2JG    0.0210
Ae52RYE    0.0191
Zg916R0    0.0138
9833QG2    0.0115
31O2pOP    0.0045
8VZ21mW    0.0040
6ZL818K   -0.0063
kW0VR8O   -0.0094
w3AQXq2   -0.0107
w3kKba1   -0.0286
aQr5AZ9   -0.0360
MdnaJk6   -0.0593
dY6ak2J   -0.0657
dtype: float64
2Xrd9ox    1.0000
zgnK7bG    0.2756
3PPK2V6    0.2071
8VZ2mPo    0.1670
eV9N0Kd    0.1379
LEw2JPM    0.0982
31O2pOP    0.0982
Y8O96KA    0.0904
9833QG2    0.0862
R9wMd3d    0.0855
gX5amYK    0.0757
6ZQdKqp    0.0526
Ae52RYE    0.0386
aQr5AZ9    0.0233
8VZ21mW    0.0143
vZXw2JG    0.0042
x2Y3xJg   -0.0017
Ae8WZqw   -0.0026
w3kKba1   -0.0130
6ZL818K   -0.0141
Zg916R0   -0.0201
kW0VR8O   -0.0